# Creating Templates

We've used templates in previous notebooks. Templates can be as simple as plain HTML but that wouldn't take advantage of many of Django's features that integrate with the rest of the framework. In this notebook, you'll learn how to show variables in templates, use filters and finally, use template tags. This is the final notebook in the series of beginner notebooks for Django.

## [Variables](https://docs.djangoproject.com/en/dev/ref/templates/language/#variables)

In Django templates, variables are shown by using double curly braces. We used a template example in the previous notebook named `'template.html'` with a variable.

Now let's try to replace the contents of that notebook with the HTML code below:

In [ ]:
%%html
<html>
<head>
  <meta charset="utf-8">
  <meta http-equiv="x-ua-compatible" content="ie=edge">
  <title>Hello world ph0wZx</title>
  <meta name="description" content="">
  <meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>

<h1>Hello world</h1>

<p>{{ message }}</p>

<p>{{ variable }}</p>
<pre>"variable" is a key in our page's context</pre>

<p>{% if object %}{{ object }}{% endif %}</p>

{% if form %}
<form method="POST" enctype="multipart/form-data">
    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit">
</form>
{% endif %}
</body>
</html>

![Web_Dev2](./images/img_hello_world2.png)

Variable names are keys in the page's context data. The corresponding value is shown when a key is accessed in the template. When a variable represents an object, a __`.`__ (dot) is used to access its attributes. When the template system encounters a dot, it tries the following lookups in this order:

1. Dictionary lookup
2. Attribute or method lookup
3. Numeric index lookup

If the resulting value is callable, it is called with no arguments. The result of the call becomes the template value.

## [Filters](https://docs.djangoproject.com/en/dev/ref/templates/language/#filters)

Variables can be modified by filters. Filters are applied to variables using a __`|`__ (pipe).

`{{ variable|lower }}`

`{{ variable|title }}`

Filters can be chained, from left to right, with the result from the filter on the left passed to filter on the right.

`{{ text|truncatewords:30|default:"no text" }}`

Many [built-in filters](https://docs.djangoproject.com/en/dev/ref/templates/builtins/#ref-templates-builtins-filters) are available. If no built-in filter fits your use case, it's easy to [make](https://docs.djangoproject.com/en/dev/howto/custom-template-tags/#writing-custom-template-filters) one.

## [Tags](https://docs.djangoproject.com/en/dev/ref/templates/language/#tags)

Template tags provide some control flow functionality, allow you to load external information, and other functionality in your templates. Some tags work alone while some come in pairs. A pair of curly braces and percent delimiters __`{% (tag) %}`__ are used by template tags.

### {% include 'template_name' %}

`Include` renders a template in the current context, useful for reusable template components. The `template_name` includes the path to the template from within one of Django's known template directories.

```
{% include 'components/navbar.html' %}
```

### {% extends 'template_name' %}

`Extends` is usually the first line of a child template and signals that it extends a parent template with the given `template_name`. Used in conjuction with block tags `{% block 'name' %}`.

## Parent template - base.html:

In [ ]:
%%html
<!DOCTYPE html>
<html lang="en">
<head>
    <link rel="stylesheet" href="style.css" />
    <title>{% block title %}My blog{% endblock %}</title>
</head>

<body>
    <div id="sidebar">
        {% block sidebar %}
        <ul>
            <li><a href="/">Home</a></li>
            <li><a href="/blog/">Blog</a></li>
        </ul>
        {% endblock %}
    </div>

    <div id="content">
        {% block content %}{% endblock %}
    </div>
</body>
</html>

## Child template - index.html:

In [ ]:
%%html
{% extends "base.html" %}

{% block title %}My Django blog!{% endblock %}

{% block content %}
  {% for entry in blog_entries %}
    <h2>{{ entry.title }}</h2>
    <p>{{ entry.body }}</p>
  {% endfor %}
{% endblock %}

### `{% block 'name' %}`

`Block` tags come as a pair together with `endblocks` and are given names. `Block` tags are used in both parent and child templates. When a child template _extends_ a parent template, the contents of a child template's blocks override the parent's blocks with the same name.

### `{% for %}`

`For` tags allow you to loop through a list, making each list item available as a variable. Paired with `endfor`, the section between `for` and `endfor` are repeated by the length of the list. The way it works is similar to a Python `for` loop.

```
{% for object in object_list %}
    {{ object.id }}. {{ object.name }}
{% endfor %}
```

### `{% if %}`

`If` evaluates the truthiness of a variable. The way it works is similar to a Python `if` statement. It works together with the `elif`, `else`, and `endif` tags.

In [ ]:
%%html
{% if name %}
    Hello, {{ name }}!
{% else %}
    Hello, guest!
{% endif %}

### `{% load %}`

`Load` allows you to load other template tags. Django has built-in template tags as well as other tags that can be loaded. Tags can easily be created but that is already outside of the notebook's scope. One commonly loaded tag is the `static` tag. Loading the `static` tag allows you to use the `static` template tag.

```
{% load static %}
<img src="{% static "images/default.jpg" %}" alt="Avatar" />
```

# Creating Forms

There are several ways to create forms in Django. Forms could be created manually by building an HMTL form and manually processing the HTTP POST request. Validators would have to be created for both input and processing and inputs have to be validated on both form and submission. That's the hard way.

Django has a forms module that handles form creation, input validation and processing. Forms can be generated in two ways.

## [ModelForm](https://docs.djangoproject.com/en/dev/topics/forms/modelforms/#django.forms.ModelForm)

Django provides an easy way to create forms from models with only a few lines using `django.forms.ModelForm`.

In [ ]:
from django.forms import ModelForm

from account.models import Profile


class ProfileModelForm(ModelForm):

    class Meta:
        model = Profile
        fields = ['name', 'photo', 'created', 'user']

By subclassing `ModelForm` and modifying the `model` and `fields` attributes of the `Meta` class, Django is able to generate a form with the given fields based on a model. `ModelForm` makes it easy to create forms so we can focus on getting our models right. For forms that are not tied to models, there is `django.forms.Form`. `ModelForm` uses `Form` to automatically generate fields.

## [forms.Form](https://docs.djangoproject.com/en/dev/topics/forms/#the-django-form-class)

For forms that are not tied to models, `django.forms.Form` can be used. To demonstrate, creating a form similar to our `Profile` model only needs a few things to be changed.

In [ ]:
from django import forms

class ProfileForm(forms.Form):
    name = forms.CharField(max_length=42)
    photo = forms.ImageField()
    created = forms.DateField()

# [Using Forms in Templates](https://docs.djangoproject.com/en/dev/topics/forms/#working-with-form-templates)

To use a form, it needs to be imported into your `views.py`, instantiated and added to the context.

In [ ]:
%%html
from .forms import ProfileModelForm

def form_view(request):
    template = "template.html"
    form = ProfileModelForm()
    context = {
        "form": form,
    }
    return render(request, template, context)

It can be accessed in the template like any other variable, except this variable is an object that generates an HTML form. We'll use the `.as_p` method to render each field surrounded in `<p>` tags.

One thing to note is that only the fields are rendered. The `<form>` tags are __not__ included, as well as any input buttons. The `<form>` tag needs to have a `method` attribute with a `POST` value. Once all of these are added, we can test our form.

In [ ]:
%%html
<form method="POST">
    {{ form.as_p }}
    <input type="submit">
</form>

## [Cross-Site Request Forgery](https://docs.djangoproject.com/en/dev/ref/csrf/)

Our form is not submitting. We are receiving a `Forbidden (403)` error from the server. This is one of Django's security features. 

Whenever communicating with a server, an HTTP method is used. Browsing typically uses the `GET` method, which is a "safe" method that does not change anything in the server. In our form, we explicitly used `method="POST"`, which is used when submitting information. There are other "unsafe" methods available when an action is meant to change something in the server but Django focuses only on `GET` and `POST`.

For security, Django has built-in Cross Site Request Forgery or CSRF protection. To get past this security feature, all we need to do is to add the `csrf_token` tag anywhere within our form. It adds a field with a randomly generated CSRF token to our form. Django will know how to handle this for us.

In [ ]:
%%html
<form method="POST">
    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit">
</form>

With that, our form can now be submitted.

## [Processing Forms](https://docs.djangoproject.com/en/dev/topics/forms/#the-view)

To process the form in our `views.py`, the form just needs to be instantiated with the submitted data as argument.

In [ ]:
    form = ProfileModelForm(request.POST or None)

By using boolean logic (`request.POST or None`) as the argument, our code will work whether or not there's submitted data. The form will instantiate with or without submitted data. For processing the submitted data, we add a conditional statement that executes a block of code if the form is valid.

In [ ]:
    if form.is_valid():
        profile = form.save(commit=False)
        profile.save()

If the form is valid, execute a block of code. The form is valid if the submitted data does not return any errors, all required inputs are provided and all inputs validated. It would need a great deal of attention to do this manually.

`forms.ModelForm`s have a save method, which calls the model's save method. However, `forms.Form`s are just plain forms unrelated to models and processing needs to be done manually. Adding the `commit=False` option does not save it to the database yet. It just tells Django that you still want to do something to this instance. To save it, the instance's save method needs to be called again like in our example.

## Updating a Record

To update an existing database record, the instance just needs to be passed to the form as a keyword argument to `instance`.

In [ ]:
    profile = get_object_or_404(Profile, pk=1)
    if profile:
        form = ProfileModelForm(request.POST or None, request.FILES or None, instance=profile)
    else:
        form = ProfileModelForm(request.POST or None, request.FILES or None)

    if form.is_valid():
        profile = form.save()

The form is generated with initial data from the given instance. Updating it is the same as processing a regular form.